# Scrape data from: https://en.m.wikipedia.org/wiki/Lists_of_The_New_York_Times_Fiction_Best_Sellers

In [1]:
# Import Libraries
from bs4 import BeautifulSoup as bs
import requests
import os
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Store the URL
nytimes_url = "https://en.m.wikipedia.org/wiki/Lists_of_The_New_York_Times_Fiction_Best_Sellers"

In [3]:
# Setup the Splinter Instance
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\ryana\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [4]:
# Visit the specific URL
browser.visit(nytimes_url)

In [5]:
# Test Cell for one instance
browser.click_link_by_partial_href(1934)


C:\Users\ryana\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:482: FutureWarning: browser.find_link_by_partial_href is deprecated. Use browser.links.find_by_partial_href instead.
  warnings.warn(


In [6]:
# Get Dataframe of table and convert it to list
html = browser.html
soup = bs(html, "html.parser")
table = pd.read_html(html)
df = table[0]
df = df.drop(columns = ["Date", "Author"])
df = df.drop_duplicates()
bookList = df["Book"].tolist()
bookList

['Anthony Adverse',
 'The Thin Man',
 'Work of Art',
 'The Oppermanns',
 'Seven Gothic Tales',
 'Tender is the Night',
 'Lamb in His Bosom',
 'Five Silver Daughters',
 'I, Claudius',
 'So Red the Rose',
 'Lost Horizon',
 'The Forty Days of Musa Dagh']

In [7]:
# Use the list to loop through the book titles
x = 0
for result in soup.find_all("a", href = True):
    print(result)
    if test[x] == result.get("title"):
        print(test[x])
        x += 1
        new_url = ("https://en.m.wikipedia.org" + result["href"])
        browser.visit(new_url)
        browser.back()
   # elif result != :
    #    x += 1
# Would it be a bad idea to just get everything and not worry about just getting the book titles?
# And just getting rid of the tables that don't provide the columns needed?


# I need to iterate through the results of the html page, and get the href link of the books to get a table from the next link.
# Created a list of all of the books on the page by reading in the table from the year page as a dataframe and converting to list.
# Will use this list to make sure only books are navigated through with splinter.
# Then, match the book titles (title attribute) with the ones in the list 
# After match is verified, get the href link to the books page, and then use splinter to navigate to that site browser.visit(newurl)
# Having issue getting past books with no href or class ="new"

<a class="mw-mf-page-center__mask" href="#"></a>


NameError: name 'test' is not defined

In [7]:
p_tags = soup.find_all('td')

htmlList = []
for each in p_tags: 
    htmlList.append(str(each.get_text()))

htmlList

['January 1',
 'Anthony Adverse',
 'Hervey Allen\n',
 'January 8\n',
 'January 15',
 'The Thin Man',
 'Dashiell Hammett\n',
 'January 22\n',
 'January 29\n',
 'February 5',
 'Work of Art',
 'Sinclair Lewis\n',
 'February 12\n',
 'February 19\n',
 'February 26\n',
 'March 5\n',
 'March 12\n',
 'March 19\n',
 'March 26',
 'Anthony Adverse',
 'Hervey Allen\n',
 'April 2',
 'The Oppermanns',
 'Lion Feuchtwanger\n',
 'April 9\n',
 'April 16',
 'Anthony Adverse',
 'Hervey Allen\n',
 'April 23\n',
 'April 30',
 'Seven Gothic Tales',
 'Isak Dinesen\n',
 'May 7',
 'Tender is the Night',
 'F. Scott Fitzgerald\n',
 'May 14',
 'Anthony Adverse',
 'Hervey Allen\n',
 'May 21',
 'Lamb in His Bosom',
 'Caroline Miller\n',
 'May 28',
 'Five Silver Daughters',
 'Louis Golding\n',
 'June 5',
 'Lamb in His Bosom',
 'Caroline Miller\n',
 'June 11\n',
 'June 18\n',
 'June 25\n',
 'July 2\n',
 'July 9\n',
 'July 16\n',
 'July 23',
 'I, Claudius',
 'Robert Graves\n',
 'July 30',
 'Anthony Adverse',
 'Hervey A

In [8]:
# This functio removes the books from the list of books, generated upon entering the year page,
# that don't have any links for the year page
#def updateBookList(bookList, htmlList):
    
for item in range(len(bookList)):
    for text in range(len(htmlList)):
        if bookList[item] == htmlList[text]:
            bookList.remove(bookList[item])
bookList

IndexError: list index out of range

In [ ]:
# Navigate throuught the website

# Loop through the years in the best sellers list
for year in range(1931,2021):
    try:
        browser.click_link_by_partial_href(year)
        
        # Get the Table and all of the information
        # First, create the Beautiful Soup object for the current page
        html = browser.html
        soup = bs(html, "html.parser")
        
        # Go through to all of the links to get more information on each book
        # for link in soup.find("table", class_ = "wikitable")

            
            # Collect the information from the table for each book
            
            # If there is no link, or the page hasn't been created yet, skip it
            
        # Go back to the main list to continue navigation
        browser.back()
        

    except:
        print("test")

In [ ]:
# Close the browser
browser.quit()

In [9]:
# Get Dataframe of table and convert it to list
html = browser.html
soup = bs(html, "html.parser")
table = pd.read_html(html)
df = table[0]
df = df.drop(columns = ["Date", "Author"])
df = df.drop_duplicates()
test = df["Book"].tolist()
test

['Anthony Adverse',
 'The Thin Man',
 'Work of Art',
 'The Oppermanns',
 'Seven Gothic Tales',
 'Tender is the Night',
 'Lamb in His Bosom',
 'Five Silver Daughters',
 'I, Claudius',
 'So Red the Rose',
 'Lost Horizon',
 'The Forty Days of Musa Dagh']

In [10]:
p_tags = soup.find_all('td')

test2 = []
for each in p_tags: 
    test2.append(str(each.get_text()))

test2

['January 1',
 'Anthony Adverse',
 'Hervey Allen\n',
 'January 8\n',
 'January 15',
 'The Thin Man',
 'Dashiell Hammett\n',
 'January 22\n',
 'January 29\n',
 'February 5',
 'Work of Art',
 'Sinclair Lewis\n',
 'February 12\n',
 'February 19\n',
 'February 26\n',
 'March 5\n',
 'March 12\n',
 'March 19\n',
 'March 26',
 'Anthony Adverse',
 'Hervey Allen\n',
 'April 2',
 'The Oppermanns',
 'Lion Feuchtwanger\n',
 'April 9\n',
 'April 16',
 'Anthony Adverse',
 'Hervey Allen\n',
 'April 23\n',
 'April 30',
 'Seven Gothic Tales',
 'Isak Dinesen\n',
 'May 7',
 'Tender is the Night',
 'F. Scott Fitzgerald\n',
 'May 14',
 'Anthony Adverse',
 'Hervey Allen\n',
 'May 21',
 'Lamb in His Bosom',
 'Caroline Miller\n',
 'May 28',
 'Five Silver Daughters',
 'Louis Golding\n',
 'June 5',
 'Lamb in His Bosom',
 'Caroline Miller\n',
 'June 11\n',
 'June 18\n',
 'June 25\n',
 'July 2\n',
 'July 9\n',
 'July 16\n',
 'July 23',
 'I, Claudius',
 'Robert Graves\n',
 'July 30',
 'Anthony Adverse',
 'Hervey A

In [11]:

for char in range(len(test)):
    for check in range(len(test2)):
        if test[char] == test2[check]:
        
            test.remove(test[char])
test

IndexError: list index out of range